## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

###TASK 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/news/news_train.txt"
TEST_PATH = "/content/drive/MyDrive/news/news_test.txt"

In [ ]:
train_data = pd.read_csv(TRAIN_PATH, sep="\t", header=None, names=['topic', 'headline', 'info'])
test_data = pd.read_csv(TEST_PATH, sep="\t", header=None, names=['topic', 'headline', 'info'])

In [ ]:
train_data.head()

In [ ]:
!pip install pymorphy2

import pymorphy2
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences

rus_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def text_preprocessing(text):
  punctuation = string.punctuation + '«' + '»' + '—'
  text = "".join(ch if ch not in punctuation else ' ' for ch in text)
  text = "".join([ch if not ch.isdigit() else ' ' for ch in text])
  words = text.split()
  tokens = []
  for word in words:
    word = morph.parse(word)[0].normal_form
    if word not in rus_stopwords:
      tokens.append(word)
  text = " ".join(tokens)
  return text

In [ ]:
for i in range(len(train_data)):
  train_data["headline"][i] = text_preprocessing(train_data["headline"][i])
  train_data["info"][i] = text_preprocessing(train_data["info"][i])

In [ ]:
train_data.head()

In [ ]:
for i in range(len(test_data)):
  test_data["headline"][i] = text_preprocessing(test_data["headline"][i])
  test_data["info"][i] = text_preprocessing(test_data["info"][i])

In [ ]:
# train_data = pd.read_csv("/content/drive/MyDrive/news/preprocessed_train.txt")
# test_data = pd.read_csv("/content/drive/MyDrive/news/preprocessed_test.txt")

In [ ]:
label_encoder = LabelEncoder()
train_data['target'] = label_encoder.fit_transform(train_data['topic'])
test_data['target'] = label_encoder.transform(test_data['topic'])

In [ ]:
# train_data.to_csv( "/content/drive/MyDrive/news/preprocessed_train.txt", index=False, encoding='utf-8-sig')
# test_data.to_csv( "/content/drive/MyDrive/news/preprocessed_test.txt", index=False, encoding='utf-8-sig')

###TASK 2

In [ ]:
from gensim.models import Word2Vec

In [ ]:
words = [text.split() for text in train_data["info"]]
w2v = Word2Vec(sentences=words, min_count=0)

In [ ]:
w2v.wv.most_similar('интернет', topn=10)  

###TASK 3

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(min_df = 10, max_df = 1., ngram_range = (1,2), max_features = 1000)
train_text = vectorizer.fit_transform(train_data["info"]).toarray()
test_text = vectorizer.transform(test_data["info"]).toarray()
train_labels = np.array(train_data['target'])
test_labels = np.array(test_data['target'])

In [ ]:
param_grid = {'C': [1, 10, 100], 'gamma': [0.01, 0.1, 1],'kernel': ['linear', 'sigmoid']}
model = SVC()
clf = GridSearchCV(model, param_grid, cv=2, verbose=3)
clf.fit(train_text, train_labels)

In [ ]:
pred = clf.predict(test_text)
print('SVC accuracy: ', accuracy_score(pred, test_labels))

In [ ]:
param_grid = {'solver':['liblinear','saga'], 'penalty': ['l1', 'l2']}
model = LogisticRegression()
clf = GridSearchCV(model, param_grid, cv=2, verbose=3)
clf.fit(train_text, train_labels)

In [ ]:
pred = clf.predict(test_text)
print('Logistic Regression accuracy: ', accuracy_score(pred, test_labels))